In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = 'KEY_ID'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'KEY'
os.environ['AWS_DEFAULT_REGION'] = 'us-west-1'

In [ ]:
!pip install boto3
!pip install s3fs

  Using cached botocore-1.42.42-py3-none-any.whl.metadata (5.9 kB)
Using cached botocore-1.42.42-py3-none-any.whl (14.6 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.42.30
    Uninstalling botocore-1.42.30:
      Successfully uninstalled botocore-1.42.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 3.1.1 requires botocore<1.42.31,>=1.41.0, but you have botocore 1.42.42 which is incompatible.


  Using cached botocore-1.42.30-py3-none-any.whl.metadata (5.9 kB)
Using cached botocore-1.42.30-py3-none-any.whl (14.6 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.42.42
    Uninstalling botocore-1.42.42:
      Successfully uninstalled botocore-1.42.42
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.42.42 requires botocore<1.43.0,>=1.42.42, but you have botocore 1.42.30 which is incompatible.


In [ ]:
#aws SDK for python that enable direct API interaction with S3
import boto3
import pandas as pd

In [ ]:
ratings = pd.read_csv(f's3://movielens-rec-bucket/raw/rating.csv')
movies = pd.read_csv(f's3://movielens-rec-bucket/raw/movie.csv')

print(ratings.head())
print(ratings[ratings['rating'] >= 4])

   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40
          userId  movieId  rating            timestamp
6              1      151     4.0  2004-09-10 03:08:54
7              1      223     4.0  2005-04-02 23:46:13
8              1      253     4.0  2005-04-02 23:35:40
9              1      260     4.0  2005-04-02 23:33:46
10             1      293     4.0  2005-04-02 23:31:43
...          ...      ...     ...                  ...
20000256  138493    66762     4.5  2009-10-17 18:50:08
20000257  138493    68319     4.5  2009-12-07 18:15:20
20000258  138493    68954     4.5  2009-11-13 15:42:00
20000259  138493    69526     4.5  2009-12-03 18:31:48
20000261  138493    70286     5.0  2009-11-13 15:42:24

[9995410 rows x 4 columns]


In [ ]:
#preprocessing data
from scipy.sparse import csr_matrix

def convert_to_implicit(ratings, threshold: float = 4.0):
  '''
  convert explicit ratings to implicit feedback.
  Rating >= threshold becomes 1 interaction
  '''

  #add interaction column to ratings df
  ratings['interaction'] = (ratings['rating'] >= threshold).astype(int)
  ratings = ratings[ratings['interaction'] > 0]
  return ratings

In [ ]:
def encoding(ratings):
  '''
  encode userId and movieId
  return mappings and encoded data
  '''
  userId = {user: id for id, user in enumerate(ratings['userId'].unique())}
  itemId = {item: id for id, item in enumerate(ratings['movieId'].unique())}

  #add new mapping column for userUd and movieId
  ratings['user_idx'] = ratings['userId'].map(userId)
  ratings['item_idx'] = ratings['movieId'].map(itemId)

  return ratings, userId, itemId

In [ ]:
def sparse_matrix(ratings, n_users: int, n_items:int) -> csr_matrix:
  '''
  building a sparse matrix(user x item) for training the ALS implicit later
  '''
  interactions = csr_matrix(
      (ratings['interaction'], (ratings['user_idx'], ratings['item_idx'])),
      shape=(n_users, n_items)
  )

  return interactions

In [ ]:
#testing to make sure the 3 functions work
ratings_imp = convert_to_implicit(ratings)
print(ratings_imp.head())

ratings_encode, userId, itemId = encoding(ratings)
print(ratings_encode.head())

sparse_matrix = sparse_matrix(ratings_encode, len(userId), len(itemId))
print(sparse_matrix.shape)

    userId  movieId  rating            timestamp  user_idx  item_idx  \
6        1      151     4.0  2004-09-10 03:08:54         0         6   
7        1      223     4.0  2005-04-02 23:46:13         0         7   
8        1      253     4.0  2005-04-02 23:35:40         0         8   
9        1      260     4.0  2005-04-02 23:33:46         0         9   
10       1      293     4.0  2005-04-02 23:31:43         0        10   

    interaction  
6             1  
7             1  
8             1  
9             1  
10            1  
   userId  movieId  rating            timestamp  user_idx  item_idx  \
0       1        2     3.5  2005-04-02 23:53:47         0         0   
1       1       29     3.5  2005-04-02 23:31:16         0         1   
2       1       32     3.5  2005-04-02 23:33:39         0         2   
3       1       47     3.5  2005-04-02 23:32:07         0         3   
4       1       50     3.5  2005-04-02 23:29:40         0         4   

   interaction  
0            0 

In [2]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp312-cp312-linux_x86_64.whl size=933265 sha256=d3399de0d0cdb6ce63634cc2295e4dca5cda77c3dd4af1fad6d24f6ab08f10d0
  Stored in directory: /root/.cache/pip/wheels/b2/00/4f/9ff8af07a0a53ac6007ea5d739da19cfe147a2df542b6899f8
Successfully built implicit
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [3]:
import implicit
import pickle

In [ ]:
def train_als(sparse_matrix, factors = 50, regularization = 0.1, iterations = 20, use_gpu: True):
  '''
  train implicit ALS model on the user_item sparse matrix.

  parameters:
  - sparse_matrix: csr_matrix, user-item interaction matrix
  - factors: int, number of latent
  - regularization: float, regularization parameter
  - iterations: int, number of iterations
  - use_gpu: bool, whether to use GPU for training

  returns:
  - model: implicit ALS model
  '''

  model = implicit.als.AlternatingLeastSquares(
      factors=factors,
      regularization=regularization,
      calculate_training_loss=True,
      iterations=iterations,
      use_gpu=use_gpu
  )

  item_user_data = sparse_matrix.T.tocsr()
  model.fit(item_user_data)

  return model


In [ ]:
def save_model_to_s3(model, filename, bucket_name):
  '''
  Save trained ALS model to local file and upload to S3
  '''

  with open(filename, "wb") as f:
    pickle.dump(model, f)

  s3 = boto3.client("s3")
  s3.upload_file(filename, bucket_name, f"models/{filename}")

In [ ]:
# Train ALS
als_model = train_als(sparse_matrix, factors=50, iterations=20, regularization=0.01)

# Generate recommendations for user 0
user_idx = 0
recommended = als_model.recommend(user_idx, sparse_matrix, N=10)

# Save model to S3
save_model_to_s3(als_model, "als_model.pkl", bucket_name="my-movielens-bucket")
